## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['Cornell', 'COPY!!!!!!!!!!!!!!!!!.ipynb', 'gpt2-gerchef', 'Copy of Fine-tune a non-English GPT-2 Model with Huggingface', 'tess_copy_commentgeneration.ipynb', 'LifeProTips', 'askredditpreprocessing.ipynb', 'UIUC', 'bigtenLinearSVC.pickle', 'model1.ipynb', 'todayilearnedgeneration.ipynb', 'commentgeneration.ipynb', 'Test_Evaluation_Metrics.ipynb.ipynb', 'preprocessCornell.ipynb']


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.2 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=2

In [ ]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/Cornell.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
# corpus = Corpus(filename=download("subreddit-Cornell"))
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [ ]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()
# type(convo_df["vectors"][0])
# test_rows = convo_df.loc[convo_df["vectors"] != []]
# len(test_rows)
# test_rows = test_rows.loc[test_rows["meta.gilded"] != 0]
# print(len(convo_df))
# given a sentence what would the next sentence be for a given post.
# 40,000 

# seems like if a utt is a post, then the id and convo_id are the same. therefore maybe
# we could iterate through convo_df, for each convo_id, and then look to utt and pull 
# all associated utts with the convo id. This should give us the post and all comments.

# https://rowlando13.medium.com/everything-gpt-2-4-data-preparation-514cb62f9f3b

In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)
temp.head()
posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]
comments.head()
len(comments)
len(posts) # 10744
# fixed method: 7735
# posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]')]
posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
# comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]

conversation_ids_in = list(posts["conversation_id"])
print(conversation_ids_in[0])
coversation_ids = []
for id in conversation_ids_in:
  dataframe = comments[comments["conversation_id"] == id]
  com_ids = list(dataframe.index)
  if com_ids != []:
    coversation_ids.append(id)

top_posts = posts.head(20000)
top_posts.head()

print(len(top_posts))
#comments.head()



Streaming output truncated to the last 5000 lines.
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in here
in he

In [ ]:
datasets_to_use = []
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

In [ ]:
import re
from nltk import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", sep_token = "<s>")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
train_ids, test_ids = train_test_split(coversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
# datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]
for index, id_list in enumerate(ids_total):
  for id in id_list:
      dataframe = comments[comments["conversation_id"] == id]
      post = top_posts.loc[id]['text']
      post = re.sub("\n", " ", post)
      post = re.sub("&gt;", "", post)
      post = re.sub("  ", " ", post)
      post = re.sub("&amp;", "", post)
      post = re.sub("#x200B;", "", post)
      post = re.sub(r'http\S+', '[url link]', post)
      post_tokens = word_tokenize(post)
      len_post = len(post_tokens)
      post_len[index].append(len_post)
      commented_here = ""
      com_ids = list(dataframe.index)
      if (len_post <= 512):
          running_sum = 0 # testing
          for id_c in com_ids:
            out_text = dataframe.loc[id_c]['text']
            out_text = str(out_text).strip()
            out_text = re.sub(r"\s", " ", out_text)
            out_text = re.sub(r"\n", " ", out_text)
            out_text = re.sub("&gt;", "", out_text) # added preprocess
            out_text = re.sub("  ", " ", out_text) # added preprocess
            out_text = re.sub("&amp;", "", out_text) # added preprocess
            out_text = re.sub("#x200B;", "", out_text) # added preprocess
            out_text = re.sub(r'http\S+', '[url link]', out_text) # added preprocess
            out_text_tokens = word_tokenize(out_text) # added this
            running_sum += len(out_text_tokens) # added this
            commented_here = commented_here + " " + out_text
            print("commented_here", commented_here)
            print("running sum", running_sum)
            if running_sum > len_post: # changed this to 512
              break # added this
          comment_tokens = word_tokenize(commented_here)
          com_len[index].append(len(comment_tokens))
          # utterance = "<bos> " + post + " <s> " + commented_here + "<eos> " # added a space behind <eos>
          post_utter = "<bos> " + post + " <eos> "
          comment_utter = "<bos>" + commented_here + " <eos> "
          print("comment_utter", comment_utter)
            # word tokenizer find average 
            #utter = tokenizer(utterance, truncation=True)
            #print(utterance)
          posts_datasets_to_use[index].append(post_utter)
          comments_datasets_to_use[index].append(comment_utter)
print(posts_datasets_to_use[0][:10])
print(comments_datasets_to_use[0][:10])
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(post_len[0][0])
print(com_len[0][0])
print(len(post_len[0]))
print(len(com_len[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Streaming output truncated to the last 5000 lines.
comment_utter <bos> The biggest difference between project teams directed towards competition and those that aren't are that competition based teams are very iteration based, you don't work on many new things and won't really have the chance to work on things from scratch and experience the design process. Otherwise, you build something completely new.  You can always accept the offer and drop if you get into another team, although that would reflect rather badly on you if you applied again later on. But as a freshman, almost any project team will be a good learning experience. If you're interested in the project team, go ahead. However, if you like another team more but you commit to the first one, you may be left wondering what if you made it onto the other. <eos> 
commented_here  Thanks! This lays things out really well for me, much appreciated.
running sum 14
commented_here  Thanks! This lays things out really well for me, much app

In [ ]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 96))
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 75))

print(averages_post)
print(boundary_post)
print(averages_com)
print(boundary_com)

[74.14974113898845, 82.47550776583034, 78.07875894988067]
[227.3199999999997, 240.0, 258.52]
[116.05102040816327, 118.33131801692866, 118.65703971119133]
[145.0, 146.0, 146.0]


In [ ]:
# print(posts_datasets_to_use[0][:10])
#print(posts_datasets_to_use[0][0])
#print(comments_datasets_to_use[0][0])
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 96))
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 75))
for index1, datasets in enumerate(post_len):
  for index, length in enumerate(datasets):
    if length > boundary_post[index1]:
      posts_removed_ind[index1].append(index)
for index1, datasets in enumerate(com_len):
  for index, length in enumerate(datasets):
    if length > boundary_com[index1]:
      coms_removed_ind[index1].append(index)
for index in range(3):
  temp_pairing = []
  temp_pairing.append(posts_removed_ind[index])
  temp_pairing.append(coms_removed_ind[index])
  our_set = set().union(*temp_pairing)
  for i in sorted(our_set, reverse=True):
    del post_len[index][i]
    del com_len[index][i]
    del posts_datasets_to_use[index][i]
    del comments_datasets_to_use[index][i]
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(len(post_len[0]))
#print(coms_removed_ind)
#print(boundary_post)
#print(boundary_com)
#print(post_len[0][0])
#print(com_len[0][0])
#print(len(tokenizer))

IndexError: ignored

In [ ]:
train_tensors = []
val_tensors = []
test_tensors = []
train_com_tensors = []
val_com_tensors = []
test_com_tensors = []
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(len(tokenizer))

50260


In [ ]:
post_tensors_data = [train_tensors, val_tensors, test_tensors]
comments_tensors_data = [train_com_tensors, val_com_tensors, test_com_tensors]
# there doesn't seem to be a way to associate the two permanently, since tokenizer automatically removes it.
# maybe the exception is you reach into the other comment and remove?
# for instance, loop with like index: if it excepts, then: you go into the other coms dataset_to_use and remove that specific index.
# say in train, which is index = 0, and we find that at index1, post is too long. go into coms_dataset_to_use and delete that entry.
# i assume that it does the same thing for the other one. 
post_max = []
com_max = []
for index, dataset_tensors in enumerate(posts_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    #try: 
    encoded_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True) # changed here
    # encoded_input = tokenizer(text, return_tensors="pt", truncation=True)
    post_max.append(encoded_input["input_ids"].shape[1]) # just checking if the truncation worked
    if encoded_input["input_ids"].shape[1] != encoded_input["attention_mask"].shape[1]:
      raise NameError("shape doesn't align for some reason")
    post_tensors_data[index].append(encoded_input)
    #except:
    #print(index1) # watch for big tokens how to. like does this already truncate? 
    #del comments_datasets_to_use[index][index1]
    #del posts_datasets_to_use[index][index1] #maybe this one too for the next pass around
      # only problem is are the indexes reliable? like does index1 mean the same thing in both when you delete
      # and does a failed tokenizer actually do anything

for index, dataset_tensors in enumerate(comments_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    #try: 
    encoded_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True) # changed here
    # encoded_input = tokenizer(text, return_tensors="pt", truncation=True)
    com_max.append(encoded_input["input_ids"].shape[1])
    if encoded_input["input_ids"].shape[1] != encoded_input["attention_mask"].shape[1]:
      raise NameError("shape doesn't align for some reason")
    comments_tensors_data[index].append(encoded_input)

#for index, dataset_tensors in enumerate(posts_datasets_to_use):
#  for index1, text in enumerate(dataset_tensors):
#    encoded_input = tokenizer(text, return_tensors="pt", padding=True)
#    tensors_data[index].append(encoded_input)

In [ ]:
print(post_tensors_data[0][0]["input_ids"])

tensor([[50257,   314,  1101,   422,   262,  5366,    11,   481, 30605,  4447,
           307,  5814,    14,   301, 22876,  1576,   329,   262,   314,   400,
         22260,  7374,    30,   314,   761,   284,  2513,   510,   262, 22638,
           290,  2513,   284,  6367,  1136,   593,    13,   220, 50258,   220,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 5

In [ ]:
### dataset.py ###
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, dataset_type: str = 'Train'):
    super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type

  def __len__(self):
    return len(self.posts)

  def get_labels(self) -> pd.Series:
    return self.comments
  
  def __getitem__(self, idx):
    # check whether it wants a batch encoding or not
    post_encoding = self.posts[idx]
    comment_encoding = self.comments[idx]
    if self.dataset_type == 'Train':
      return_dict = {
        'input_ids': post_encoding.input_ids,
        'attention_mask': post_encoding.attention_mask,
        'labels': comment_encoding.input_ids
      }
    #else:
      #return_dict = {
        #'input_ids': post_encoding.input_ids,
        #'target_text': target_text,
        #'input_text': input_text
      #}
    return return_dict

In [ ]:
print(post_tensors_data[0][0])
print("break")
print(post_tensors_data[0][0]['input_ids'])
print("break")
print(post_tensors_data[0][0]['attention_mask'])

{'input_ids': tensor([[50257,   314,  1101,   422,   262,  5366,    11,   481, 30605,  4447,
           307,  5814,    14,   301, 22876,  1576,   329,   262,   314,   400,
         22260,  7374,    30,   314,   761,   284,  2513,   510,   262, 22638,
           290,  2513,   284,  6367,  1136,   593,    13,   220, 50258,   220,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 5

In [ ]:
#print(len(tensors_data[index][0]["input_ids"][0]))
#print(len(tensors_data[index][0]["attention_mask"][0]))
#print(encoded_input["input_ids"].shape[1])
#print(encoded_input["attention_mask"].shape[1])
post_max_np = np.array(post_max)
print(post_max_np.shape)
print(np.max(post_max_np))
com_max_np = np.array(com_max)
print(com_max_np.shape)
print(np.max(com_max_np))
print(post_tensors_data[0][0])

#train_to = zip(post_tensors_data[0], comments_tensors_data[0])
train_data = RedditDataset(post_tensors_data[0], comments_tensors_data[0])
#val_to = zip(post_tensors_data[1], comments_tensors_data[1])
val_data = RedditDataset(post_tensors_data[1], comments_tensors_data[1])
#test_to = zip(post_tensors_data[2], comments_tensors_data[2])
test_data = RedditDataset(post_tensors_data[2], comments_tensors_data[2])
#print(len(comments_tensors_data[0]))
#print(len(posts_datasets_to_use[0]))
#print(len(comments_datasets_to_use[0]))

(6656,)
256
(6656,)
256
{'input_ids': tensor([[50257,   314,  1101,   422,   262,  5366,    11,   481, 30605,  4447,
           307,  5814,    14,   301, 22876,  1576,   329,   262,   314,   400,
         22260,  7374,    30,   314,   761,   284,  2513,   510,   262, 22638,
           290,  2513,   284,  6367,  1136,   593,    13,   220, 50258,   220,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259

In [ ]:
print(type(test_data))
print(type(test_data).__getitem__(test_data, idx = 0))

<class '__main__.RedditDataset'>
{'input_ids': tensor([[50257,   314,  1101,  2111,   284,  4351,   284, 11054, 14003,   422,
          1052,   290,   311,   290,   314,  1549,  1842,   617,  5608,     0,
         40571,   502,   379, 34681,  1219,  1073,  3400,    16,  3387,     0,
          1002,   345,   991,   467,   994,  3863,   356,   714,  1561,   625,
          9965,   393,  1223,    11,   314,  1549,  1842,   284,  2298,   534,
          3632,    13,   220,   220, 50258,   220, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
     

In [ ]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
#from transformers import TextDataset
#print(len(train_dataset_to_use))
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints-preprocess")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    # do_train=True,
    num_train_epochs=1, # change this back to 2 
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy = "steps", #comment this
    eval_steps = 200,
    save_steps= 800,
    warmup_steps = 200,
    load_best_model_at_end = True, #comment this
    prediction_loss_only = True,
    )
# try and fix the above params as well
# convert dataset_to_use from a dictionary to a dataset type so that we can use it in the trainer
train_dataset_to_use_converted = torch.utils.data.DataLoader(train_dataset_to_use)
#print(type(train_dataset_to_use_converted))
test_dataset_to_use_converted = torch.utils.data.DataLoader(test_dataset_to_use)

#early_stop = EarlyStoppingCallback(5, 1.0) #comment this

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    #train_dataset=train_dataset_up,
    #eval_dataset=test_dataset_up,
    #train_dataset=train_dataset_to_use_converted,
    #eval_dataset=test_dataset_to_use_converted,
    train_dataset=train_data,
    eval_dataset=test_data
    #callbacks = [early_stop] #comment this
)

# depends on whether we want to use Trainer or not. I think using Trainer would be better if we get it to work,
# but I'm not sure how feasible it is. Additionally manual is kinda what we did in class
# ask about batching our data, and ask about num_labels. Batching our data in that dataloader part. like how to iterate over batches of data.
# idk if need to batch

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
200,No log,4.152970
400,No log,3.725489
600,11.918800,3.495439
800,11.918800,3.463169
1000,3.631200,3.448711
1200,3.631200,3.403832
1400,3.631200,3.370225
1600,3.514900,3.340180
1800,3.514900,3.318264
2000,3.456800,3.304970


TrainOutput(global_step=4998, training_loss=4.266419360713035, metrics={'train_runtime': 2808.3533, 'train_samples_per_second': 1.78, 'train_steps_per_second': 1.78, 'total_flos': 652968787968000.0, 'train_loss': 4.266419360713035, 'epoch': 1.0})

In [ ]:
trainer.save_model()

In [ ]:
predictions = []
ALLOW_NEW_LINES = True
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
start = "I am at "
start_with_bos = "<bos>" + start
encoded_prompt = tokenizer(start_with_bos, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160,
    min_length=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=10
    )
generated_sequences = []
print(output_sequences)
# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())
                
    for i, g in enumerate(generated_sequences):
        predictions.append([start, g])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,  2061,   318,  ...,   220,   220,   220],
        [50257,  2061,   318,  ...,   220,   220,   220],
        [50257,  2061,   318,  ...,   220,   220,   220],
        ...,
        [50257,  2061,   318,  ...,   220,   220,   220],
        [50257,  2061,   318,  ...,   220,   220,   220],
        [50257,  2061,   318,  ...,   220,   220,   220]], device='cuda:0')


In [ ]:
for x in predictions:
    print(x)

['What is the best english class?', "What is the best english class? I used it when i was at a business school, and I tried to understand most of the language in English and I just don't think that I can get the English experience anywhere. Anyone who has taken it and has any experience with the class can attest to the fact that I could really get in at the most advanced language in the class. How did that class go and why?"]
['What is the best english class?', "What is the best english class? I used it when i was at a business school, and I tried to understand most of the language in English and I just don't think that I can get the English experience anywhere. Anyone who has taken it and has any experience with the class can attest to the fact that I could really get in at the most advanced language in the class. How did that class go and why?"]
['What is the best english class?', 'What is the best english class? Do they have an external link for it?']
['What is the best english clas

In [ ]:
from transformers import pipeline
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints-preprocess")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
generator = pipeline('text-generation', model=OUTPUT_DIR, tokenizer=tokenizer)
output_sequences = generator("I am locked in the cupboard above the coffee shop in Ithaca. Help.", max_length=200, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
output_sequences = generator("what is the best ", max_length=200, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
print(output_sequences)

[{'generated_text': "I am locked in the cupboard above the coffee shop in Ithaca. Help. The rest happens as happens. I've been trying getting tickets since school started, but I didn't have any. What do I do now? I'm scared.                                                                                                                                                   "}, {'generated_text': 'I am locked in the cupboard above the coffee shop in Ithaca. Help.                                                                                                                                                                                    '}, {'generated_text': 'I am locked in the cupboard above the coffee shop in Ithaca. Help.                                                                                                                                                                                    '}, {'generated_text': 'I am locked in the cupboard above the coffee shop in Ithaca. Help

In [ ]:
output_sequences = trainer.model.generate(
    input_ids=post_tensors_data[1][2]["input_ids"],
    max_length=160,
    min_length=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: ignored

In [ ]:
### DO NOT RUN THIS CELL ###
# nltk.download('punkt')
#dataset_to_use =[]
#for id in conversation_ids:
#  dataframe = comments[comments["conversation_id"] == id]
#  post = posts.loc[id]['text']
#  output = dataframe.head(5)
#  commented_here = ""
#  com_ids = list(output.index)
#  for id_c in com_ids:
#    out_text = output.loc[id_c]['text']
    #preprocess outtext
    # https://colab.research.google.com/drive/1ebqk8kP60GHPzT7aSqbzgacvRKgISqZm#scrollTo=V36gOIOfLHvB
    # shouldn't we also apply the preprocessing to the post
#    out_text = str(out_text).strip()
#    out_text = re.sub(r"\s", " ", out_text)
#    commented_here = commented_here + " " + out_text

#  utterance = "<|endoftext|> " + post + " <|Convo|> " + commented_here + "<|endoftext|>"
  # need to account if too long
  # print("utterance ", len(utterance))
#  utter = tokenizer(utterance, truncation=True)
  # print("utter ", len(utter))
  # print(utter)
  # print(utter)
  # if len(utter) > 1024:
    # print("too long ", utter)
#  dataset_to_use.append(utter)
  #utter = GPT2Tokenizer(utterance)
  #print(utter)
  #gpt-2 1024 max
  #if len(utter) > 1024:

#print(out_data["text"][3] == '')
#top_posts = out_data.head(8000)
#top_posts.head()
# print(len(top_posts))


In [ ]:
#print(len(dataset_to_use))
#train_in, test_in_one = train_test_split(dataset_to_use, test_size=0.25)
#print(len(train_in))
#print(len(test_in_one))
#print(len(train_in) + len(test_in_one))

In [ ]:
#val_in, test_in = train_test_split(test_in_one, test_size=0.5)

In [ ]:
# saving the tokenized data
#GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
#TRAIN_TOKENIZER_PATH = os.path.join(GOOGLE_DRIVE_PATH, "tokenizeddata", "cornelltraintokenized.pickle")
#VAL_TOKENIZER_PATH = os.path.join(GOOGLE_DRIVE_PATH, "tokenizeddata", "cornellvaltokenized.pickle")
#TEST_TOKENIZER_PATH = os.path.join(GOOGLE_DRIVE_PATH, "tokenizeddata", "cornelltesttokenized.pickle")

In [ ]:
# print(TOKENIZER_PATH)
#pickle.dump(train_dataset_to_use, open(TRAIN_TOKENIZER_PATH, "wb"))
#pickle.dump(val_dataset_to_use, open(VAL_TOKENIZER_PATH, "wb"))
#pickle.dump(test_dataset_to_use, open(TEST_TOKENIZER_PATH, "wb"))

In [ ]:
#print(train_dataset_to_use[:10])
#train_dataset_to_use = []
#print(train_dataset_to_use)
#train_dataset_to_use = pickle.load(open(TRAIN_TOKENIZER_PATH, "rb"))
#print(train_dataset_to_use[:10])

In [ ]:
print(train_dataset_to_use[:10])
print(type(train_dataset_to_use))
print(len(train_dataset_to_use))
print(train_dataset_to_use[0])


# not sure if the length is correct, though it might be 
# input_ids represents a mapping of a word to gpt-2's internal vocabulary, and attention mask corresponds to whether you should consider a specific word in whole context, the importance 

["<bos> I'm a freshman in ILR who was wondering when people on dean's list typically. get notified.  I'm pretty sure I made it after looking at the GPA/credit requirements, but have not yet been notified.  Thanks!  <s>  For engineering they get sent in mid to late January. May be a while, depending on how your registrar does things, but it seem like they do it quite soon after the semester ends to rank ILRies. For engineering, I didn't get the notification for my first fall semester until the following June, but it was already denoted on my transcript.   ILR seems to have pretty set guidelines for dean's list: GPA minimum (not rounded), 12 letter graded credits, successful completion of all courses. If you meet those, you'll eventually get the line on your transcript. Also, dean's list as a distinction is kind of meaningless, but I guess it's a thing to have. I'm in CALS and I received my notification about a week ago? Give or take a few days. (Break has altered my perspective of time)

In [ ]:
# saving the tokenized data
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
TRAIN_PATH = os.path.join(GOOGLE_DRIVE_PATH, "nontokenizeddata/train", "cornelltrain.txt")
VAL_PATH = os.path.join(GOOGLE_DRIVE_PATH, "nontokenizeddata/val", "cornellval.txt")
TEST_PATH = os.path.join(GOOGLE_DRIVE_PATH, "nontokenizeddata/test", "cornelltest.txt")
print(TRAIN_PATH)

drive/My Drive/448 Group: Jedi Order/models/nontokenizeddata/train/cornelltrain.txt


In [ ]:
with open(TRAIN_PATH, "w", encoding="utf-8") as f_1:
    for tra in train_dataset_to_use:
        f_1.write(tra)
f_1.close()
with open(TEST_PATH, "w", encoding="utf-8") as f_2:
    for tes in test_dataset_to_use:
        f_2.write(tes)
f_2.close()
with open(VAL_PATH, "w", encoding="utf-8") as f_3:
    for val in val_dataset_to_use:
        f_3.write(val)
f_3.close()

https://huggingface.co/transformers/v3.4.0/custom_datasets.html


In [ ]:
# https://www.kaggle.com/code/baekseungyun/gpt-2-with-huggingface-pytorch


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
def tok(example):
  print(type(example['text']))
  print(len(example['text']))
  assert False
  encodings = tokenizer(example['text'], truncation=True, return_tensors='pt', padding=True)
  return encodings

In [ ]:
print(train_tokenized["train"]['input_ids'][0])

[50257, 314, 1101, 257, 18621, 287, 14639, 49, 508, 373, 11263, 618, 661, 319, 34798, 338, 1351, 6032, 13, 651, 17600, 13, 220, 314, 1101, 2495, 1654, 314, 925, 340, 706, 2045, 379, 262, 45283, 14, 43082, 5359, 11, 475, 423, 407, 1865, 587, 17600, 13, 220, 6930, 0, 50259, 1890, 8705, 484, 651, 1908, 287, 3095, 284, 2739, 3269, 13, 1737, 307, 257, 981, 11, 6906, 319, 703, 534, 4214, 20040, 857, 1243, 11, 475, 340, 1283, 588, 484, 466, 340, 2407, 2582, 706, 262, 24878, 5645, 284, 4279, 14639, 49, 444, 13, 1114, 8705, 11, 314, 1422, 470, 651, 262, 14483, 329, 616, 717, 2121, 24878, 1566, 262, 1708, 2795, 11, 475, 340, 373, 1541, 2853, 5191, 319, 616, 14687, 13, 220, 220, 14639, 49, 2331, 284, 423, 2495, 900, 9949, 329, 34798, 338, 1351, 25, 45283, 5288, 357, 1662, 19273, 828, 1105, 3850, 44545, 10824, 11, 4388, 11939, 286, 477, 10902, 13, 1002, 345, 1826, 883, 11, 345, 1183, 4191, 651, 262, 1627, 319, 534, 14687, 13, 4418, 11, 34798, 338, 1351, 355, 257, 12941, 318, 1611, 286, 24513, 11, 

In [ ]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 499168 KiB | 499168 KiB | 499168 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |      0 B   |
|       from small pool |  12768 KiB |  12768 KiB |  12768 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 499168 KiB | 499168 KiB | 499168 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |

In [ ]:
#print(model)
#print(model.config.vocab_size)
#print(len(tokenizer))

#model.resize_token_embeddings(len(tokenizer))
#print(model.config.vocab_size)
#print(train_tokenized["train"]["input_ids"][0])
print(type(train_tokenized["train"]["input_ids"][0]))

<class 'list'>


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 499168 KiB | 499168 KiB | 499168 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |      0 B   |
|       from small pool |  12768 KiB |  12768 KiB |  12768 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 499168 KiB | 499168 KiB | 499168 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


RuntimeError: ignored

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
trainer.save_model()

In [ ]:
## Cell not in use ##
# model = AutoModel.from_pretrained('gpt2')
# # model = GPT2LMHeadModel.from_pretrained('gpt2')

# tokenizer = AutoTokenizer.from_pretrained('gpt2')
# # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# input_prompt = "Hello I am"
# input_ids = tokenizer.encode(input_prompt, return_tensors='pt')
# output = model.generate(input_ids, max_length=50, do_sample=True)
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generated_text)